# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']
F. Walter  ->  F. Walter  |  ['F. Walter']
C. Larkin  ->  C. Larkin  |  ['C. Larkin']
M. Ramirez-Tannus  ->  M. Ramirez-Tannus  |  ['M. Ramirez-Tannus']
J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
T. Trifonov  ->  T. Trifonov  |  ['T. Trifonov']
R. Burn  ->  R. Burn  |  ['R. Burn']
Arxiv has 64 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2402.16942


extracting tarball to tmp_2402.16942...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2402.16942/f1000w_source_nircamdark.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'table1_v4' from 'tmp_2402.16942/table1_v4.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure Fig/TIC169249234/rv_phase.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[width=.49\textwidth]{Fig/TIC169249234/rv_phase.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure Fig/TIC332558858/rv_ts.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[width=\textwidth]{Fig/TIC332558858/rv_ts.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure Fig/TIC237922465/rv_ts.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[width=\textwidth]{Fig/TIC237922465/rv_ts.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure Fig/TIC169249234/rv_ts.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[width=\textwidth]{Fig/TIC169249234/rv_ts.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 87 bibliographic references in tmp_2402.17592/WINE_main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.16942-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.16942) | **A NIRCam-dark galaxy detected with the MIRI/F1000W filter in the  MIDIS/JADES Hubble Ultra Deep Field**  |
|| P. G. Pérez-González, et al. -- incl., <mark>L. Boogaard</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2024-02-28*|
|*Comments*| *Submitted to ApJL, comments welcome*|
|**Abstract**| We report the discovery of Cerberus, an extremely red object detected with the MIRI Deep Imaging Survey (MIDIS) observations in the F1000W filter of the Hubble Ultra Deep Field. The object is detected at S/N~6, with F1000W~27 mag, and it is extremely faint in both the NIRCam data gathered by the JWST Advanced Deep Extragalactic Survey, JADES, with ~30.5 mag $5\sigma$ upper limits in individual bands, as well as in the MIDIS F560W ultra deep data ($\sim$29 mag, $5\sigma$). Analyzing the spectral energy distribution built with individual (low S/N) optical-to-mid-infrared filters and (S/N~5) stacks, we discuss the possible nature of this red NIRCam-dark source using a battery of codes. We discard the possibility of Cerberus being a Solar System body based on the $<$0.016" proper motion in the 1-year apart JADES and MIDIS observations. A sub-stellar Galactic nature is deemed unlikely, given that the Cerberus' relatively flat NIRCam-to-NIRCam and very red NIRCam-to-MIRI flux ratios are not consistent with any brown dwarf model. The extragalactic nature of Cerberus offers 3 possibilities: (1) A $z\sim0.4$ galaxy with strong emission from polycyclic aromatic hydrocarbons; the very low inferred stellar mass, $\mathrm{M}_\star=10^{5-6}$ M$_\odot$, makes this possibility highly improbable. (2) A dusty galaxy at $z\sim4$ with an inferred stellar mass $\mathrm{M}_\star\sim10^{8}$ M$_\odot$. (3) A galaxy with observational properties similar to those of the reddest little red dots discovered around $z\sim7$, but Cerberus lying at $z\sim15$, presenting a spectral energy distribution in the rest-frame optical dominated by emission from a dusty torus or a dusty starburst. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.16987-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.16987) | **X-Shooting ULLYSES: Massive Stars at low metallicity II. DR1: Advanced  optical data products for the Magellanic Clouds**  |
|| H. Sana, et al. -- incl., <mark>C. Larkin</mark>, <mark>M. Ramirez-Tannus</mark> |
|*Appeared on*| *2024-02-28*|
|*Comments*| *22 pages, 19 figures; accepted for publication in Astronomy & Astrophysics. Links to online tables and databases will be included upon publication*|
|**Abstract**| Using the medium resolution spectrograph X-shooter, spectra of 235 OB and Wolf-Rayet (WR) stars in sub-solar metallicity environments have been secured. [...]This second paper focuses on the optical observations of 232 Magellanic Clouds targets. It describes the uniform reduction of the UVB (300 - 560 nm) and VIS (550 - 1020 nm) XShootU data as well as the preparation of advanced data products [...] . The data reduction of the raw data is based on the ESO CPL X-shooter pipeline. We paid particular attention to the determination of the response curves [...] We implemented slit-loss correction, absolute flux calibration, (semi-)automatic rectification to the continuum, and a correction for telluric lines. The spectra of individual epochs were corrected for the barycentric motion, re-sampled and co-added, and the spectra from the two arms were merged into a single flux calibrated spectrum covering the entire optical range with maximum signal-to-noise ratio. [...] We provide three types of data products: (i) two-dimensional spectra for each UVB and VIS exposure; (ii) one-dimensional UVB and VIS spectra before and after response-correction, as well as after applying various processing, including absolute flux calibration, telluric removal, normalisation and barycentric correction; and (iii) co-added flux-calibrated and rectified spectra over the full optical range, for which all available XShootU exposures were combined. For many of the targets, the final signal-to-noise ratio per resolution element is above 200 in both the UVB and the VIS co-added spectra. The reduced data and advanced scientific data products will be made available to the community upon publication of this paper. [...] |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.17592-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.17592) | **Three Warm Jupiters around Solar-analog stars detected with TESS**  |
|| <mark>J. Eberhardt</mark>, et al. -- incl., <mark>T. Trifonov</mark>, <mark>R. Burn</mark> |
|*Appeared on*| *2024-02-28*|
|*Comments*| **|
|**Abstract**| We report the discovery and characterization of three giant exoplanets orbiting solar-analog stars, detected by the \tess space mission and confirmed through ground-based photometry and radial velocity (RV) measurements taken at La Silla observatory with \textit{FEROS}. TOI-2373\,b is a warm Jupiter orbiting its host star every $\sim$ 13.3 days, and is one of the two most massive known exoplanet with a precisely determined mass and radius around a star similar to the Sun, with an estimated mass of m$_p$ = $9.3^{+0.2}_{-0.2}\,M_{\mathrm{jup}}$, and a radius of $r_p$ = $0.93^{+0.2}_{-0.2}\,R_{\mathrm{jup}}$. With a mean density of $\rho = 14.4^{+0.9}_{-1.0}\,\mathrm{g\,cm}^{-3}$, TOI-2373\,b is among the densest planets discovered so far. TOI-2416\,b orbits its host star on a moderately eccentric orbit with a period of $\sim$ 8.3 days and an eccentricity of $e$ = $0.32^{+0.02}_{-0.02}$. TOI-2416\,b is more massive than Jupiter with $m_p$ = 3.0$^{+0.10}_{-0.09}\,M_{\mathrm{jup}}$, however is significantly smaller with a radius of $r_p$ = $0.88^{+0.02}_{-0.02},R_{\mathrm{jup}}$, leading to a high mean density of $\rho = 5.4^{+0.3}_{-0.3}\,\mathrm{g\,cm}^{-3}$. TOI-2524\,b is a warm Jupiter near the hot Jupiter transition region, orbiting its star every $\sim$ 7.2 days on a circular orbit. It is less massive than Jupiter with a mass of $m_p$ = $0.64^{+0.04}_{-0.04}\,M_{\mathrm{jup}}$, and is consistent with an inflated radius of $r_p$ = $1.00^{+0.02}_{-0.03}\,R_{\mathrm{jup}}$, leading to a low mean density of $\rho = 0.79^{+0.08}_{-0.08}\,\mathrm{g\,cm}^{-3}$. The newly discovered exoplanets TOI-2373\,b, TOI-2416\,b, and TOI-2524\,b have estimated equilibrium temperatures of $860^{+10}_{-10}$ K, $1080^{+10}_{-10}$ K, and $1100^{+20}_{-20}$ K, respectively, placing them in the sparsely populated transition zone between hot and warm Jupiters. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2402.16942.md
    + _build/html/tmp_2402.16942/./midis000714_nircam.v3.0.png
    + _build/html/tmp_2402.16942/./midis000714_nircam.convolved.v3.0.png
    + _build/html/tmp_2402.16942/./midis000714_nircam_redstacks.convolved.v3.0.png
    + _build/html/tmp_2402.16942/./midis000714_nircam_redstacksbis.convolved.v3.0.png
    + _build/html/tmp_2402.16942/./midis000714_nircam_bluestacks.convolved.v3.0.png
    + _build/html/tmp_2402.16942/./models_nircamdarkf1000wsource_v3.png
    + _build/html/tmp_2402.16942/./sed_nircamdarkf1000wsource_v3.png
exported in  _build/html/2402.16987.md
    + _build/html/tmp_2402.16987/./FIGURES/fluxCompare.png
    + _build/html/tmp_2402.16987/./FIGURES/QCXSH-U_2dFS-163_COADD1D_UVBA_NORM_bis.png
    + _build/html/tmp_2402.16987/./FIGURES/QCXSH-U_2dFS-163_COADD1D_UVBB_NORM_bis.png
    + _build/html/tmp_2402.16987/./FIGURES/QCXSH-U_2dFS-163_COADD1D_VISA_NORM_bis.png
    + _build/html/tmp_2402.16987/./FIGURES/QCXSH-U_2dFS-163_COADD1D_VISB_NO

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# A NIRCam-dark galaxy detected with the MIRI/F1000W filter in the MIDIS/JADES Hubble Ultra Deep Field

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2402.16942-b31b1b.svg)](https://arxiv.org/abs/2402.16942)<mark>Appeared on: 2024-02-28</mark> -  _Submitted to ApJL, comments welcome_

</div>
<div id="authors">

P. G. Pérez-González, et al. -- incl., <mark>L. Boogaard</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of $_ Cerberus_$ , an extremely red object detected with the MIRI Deep Imaging Survey (MIDIS) observations in the F1000W filter of the Hubble Ultra Deep Field. The object is detected at $S/N\sim6$ , with $\mathrm{F1000W}\sim27$ mag, and it is extremely faint in both the NIRCam data gathered by the JWST Advanced Deep Extragalactic Survey, JADES, with $\sim$ 30.5 mag $5\sigma$ upper limits in individual bands, as well as in the MIDIS F560W ultra deep data ( $\sim$ 29 mag, $5\sigma$ ). Analyzing the spectral energy distribution built with individual (low $S/N$ ) optical-to-mid-infrared filters and ( $S/N\sim5$ ) stacks, we discuss the possible nature of this red NIRCam-dark source using a battery of codes. We discard the possibility of $_ Cerberus_$ being a Solar System body based on the $<0\farcs016$ proper motion in the 1-year apart JADES and MIDIS observations. A sub-stellar Galactic nature is deemed unlikely, given that the $_ Cerberus'_$ relatively flat NIRCam-to-NIRCam and very red NIRCam-to-MIRI flux ratios are not consistent with any brown dwarf model. The extragalactic nature of $_ Cerberus_$ offers 3 possibilities: (1) A $z\sim0.4$ galaxy with strong emission from polycyclic aromatic hydrocarbons; the very low inferred stellar mass, $\mathrm{M}_\star=10^{5-6}$ M $_\odot$ , makes this possibility highly improbable. (2) A dusty galaxy at $z\sim4$ with an inferred stellar mass $\mathrm{M}_\star\sim10^{8}$ M $_\odot$ . (3) A galaxy with observational properties similar to those of the reddest little red dots discovered around $z\sim7$ , but $_ Cerberus_$ lying at $z\sim15$ , presenting a spectral energy distribution in the rest-frame optical dominated by emission from a dusty torus or a dusty starburst.

</div>

<div id="div_fig1">

<img src="tmp_2402.16942/./midis000714_nircam.v3.0.png" alt="Fig3.1" width="20%"/><img src="tmp_2402.16942/./midis000714_nircam.convolved.v3.0.png" alt="Fig3.2" width="20%"/><img src="tmp_2402.16942/./midis000714_nircam_redstacks.convolved.v3.0.png" alt="Fig3.3" width="20%"/><img src="tmp_2402.16942/./midis000714_nircam_redstacksbis.convolved.v3.0.png" alt="Fig3.4" width="20%"/><img src="tmp_2402.16942/./midis000714_nircam_bluestacks.convolved.v3.0.png" alt="Fig3.5" width="20%"/>

**Figure 3. -** In the top row, we show postage stamps of _ Cerberus_ in all NIRCam bands from the JADES DR2  ([Eisenstein, Johnson and Robertson 2023]()) . The second row shows the same stamps but convolved with a 2-pixel radius tophat filter. The third row shows incremental stacks of NIRCam filters, starting from the reddest two bands (F480M and F460M) and adding one band at a time, down to F210M. The fourth row shows similar stacks, but avoiding F460M, which presents an artifact (see main text for details). The fifth row shows stacks starting from the bluest bands (blue\_stack01 including F090W and F115W) and adding the next filter in subsequent images. All postage stamps are 1$\arcsec$$\times$1$\arcsec$, with the source marked with a circle of radius 0$\farcs$2. Green circles show the PSF FWHM (including the tophat filter). (*fig:stamps*)

</div>
<div id="div_fig2">

<img src="tmp_2402.16942/./models_nircamdarkf1000wsource_v3.png" alt="Fig6" width="100%"/>

**Figure 6. -** Detailed analysis of the spectral energy distribution of _ Cerberus_ for the main redshift solutions. The symbols are the same as described in Figure \ref{fig:photoz}. We provide best-fitting models provided by different codes that include both stellar and AGN emission.
The top-left panel shows the best-fitting brown dwarf template from the ATMO2020++ models, for a Y-type dwarf with an effective temperature of $\sim 300$K at a distance of $\sim 300$ pc.
The rest of the panels show results for $z=0.4$(top-right), $z=3.5$(bottom-left), and $z=15$(bottom-right). Main physical properties inferred with the modeling of the spectral energy distributions assuming an extragalactic origin are provided in Table \ref{tab:sed_properties}. (*fig:sps*)

</div>
<div id="div_fig3">

<img src="tmp_2402.16942/./sed_nircamdarkf1000wsource_v3.png" alt="Fig5" width="100%"/>

**Figure 5. -** _ Left:_ Spectral energy distribution of _ Cerberus_. Dots show flux measurements with $S/N>2$ for individual NIRCam and MIRI bands (filter widths shown as horizontal bars), triangles depict 1$\sigma$ upper limits. Shaded regions stand for flux measurements and filter widths for NIRCam stacks in the LW (actual $S/N>5$ measurements in most stacks) and SW (upper limits in all of them) channels. Two sets of NIRCam stacks are considered: two incremental stacks starting from the red and blue ends, respectively, and stopping at $\sim$2 $\mu$m; and 4 disjoint stacks, combining filters in groups of 2, 3, 4, and 5, with no filter in common between each set. The spectral energy distribution has been fitted with a battery of codes to estimate a photometric redshift. Most probable values and uncertainties are provided in the figure (for {\sc eazy} we provide 2 solutions since their peak probabilities are very similar), with low-z solutions plotted as thin lines, and high-z models as thick lines. _ Right:_ Photometric redshift probability distribution function for the modeling shown in the left panel. All distributions are normalized to unity at the peak. We show the main solutions, including all values provided in the spectral energy distribution plot and also secondary solutions at $z<10$ and $z>10$. (*fig:photoz*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2402.16942"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\cms}{cm s^{-2}}$
$\newcommand{\ergscm}{erg s^{-1} cm^{-2}}$
$\newcommand{\ergs}{erg s^{-1}}$
$\newcommand{\lsol}{L_{\odot}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\Lsun}{L_{\odot}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\msunyr}{M_{\odot} yr^{-1}}$
$\newcommand{\msol}{M_{\odot}}$
$\newcommand{\rsol}{R_{\odot}}$
$\newcommand{\rsun}{R_{\odot}}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\teff}{T_\mathrm{eff}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\mdot}{M_\odot}$
$\newcommand{\vinf}{v_\infty}$
$\newcommand{\vsini}{\varv \sin i}$
$\newcommand{\ha}{H{\alpha}}$
$\newcommand{\halpha}{H{\alpha}}$
$\newcommand{\hb}{H{\beta}}$
$\newcommand{\hbeta}{H{\beta}}$
$\newcommand{\hdelta}{H{\delta}}$
$\newcommand{\hgamma}{H{\gamma}}$
$\newcommand{\l}{\lambda}$
$\newcommand{\hea}{\ion{He}{i}}$
$\newcommand{\heb}{\ion{He}{ii}}$
$\newcommand{\snr}{S/N}$
$\newcommand{\xsh}{X-shooter}$
$\newcommand{\xshooter}{X-shooter}$
$\newcommand{\ullyses}{{\sc ULLYSES}}$
$\newcommand{\xshootu}{XShootU}$
$\newcommand{\sci}{{\sc science}}$
$\newcommand{\flat}{{\sc ff}}$
$\newcommand{\ff}{{\sc ff}}$
$\newcommand{\raw}{{\sc raw}}$
$\newcommand{\flux}{{\sc flux}}$
$\newcommand{\science}{{\sc science}}$
$\newcommand{\errs}{{\sc errs}}$
$\newcommand{\qual}{{\sc qual}}$</div>



<div id="title">

# X-Shooting ULLYSES: Massive Stars at low metallicity$\thanks{Based on observations collected at the European Southern Observatory under ESO program ID 106.211Z.001.}$$^,$$\thanks{Tables \ref{t:targets}, \ref{t:journal} and \ref{t:RV_results} are only available in electronic form at the CDS via anonymous ftp to cdsarc.u-strasbg.fr (130.79.128.5) or via \url{http://cdsweb.u-strasbg.fr/cgi-bin/qcat?J/A+A/}.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2402.16987-b31b1b.svg)](https://arxiv.org/abs/2402.16987)<mark>Appeared on: 2024-02-28</mark> -  _22 pages, 19 figures; accepted for publication in Astronomy & Astrophysics. Links to online tables and databases will be included upon publication_

</div>
<div id="authors">

H. Sana, et al. -- incl., <mark>C. Larkin</mark>

</div>
<div id="abstract">

**Abstract:** The $\xshootu$ project aims to obtain ground-based optical to near-infrared spectroscopy of all targets observed by the $_ Hubble Space Telescope (HST)_$ under the Director's Discretionary program $\ullyses$ .  Using the medium resolution spectrograph $\xshooter$ , spectra of 235 OB and Wolf-Rayet (WR) stars  in sub-solar metallicity environments have been secured. The bulk of the targets belong to the Large and Small Magellanic Clouds, with the exception of three stars in NGC 3109 and Sextans A. This second paper of the series focuses on the optical  observations of Magellanic Clouds targets. It describes the uniform reduction of the UVB ( $300-560$ nm) and VIS ( $550-1020$ nm) $\xshootu$ data  as well as the preparation of advanced data products that are suitable for homogeneous scientific analyses. The data reduction of the $\raw$ data is based on  the ESO CPL $\xshooter$ pipeline. We paid particular attention to the determination of the response curves. This required equal flat-fielding of the science and flux standard star data and the derivation of improved flux standard models.   The pipeline products were then processed with our own set of routines to produce a series of advanced data products.  In particular, we  implemented slit-loss correction, absolute flux calibration, (semi-)automatic rectification to the continuum, and a correction for telluric lines. The spectra of individual epochs were further corrected for the barycentric motion, re-sampled and co-added, and the spectra from the two arms were merged into a single flux calibrated spectrum covering the entire optical range with maximum signal-to-noise ratio. We identify and describe an undocumented recurrent ghost visible on the $\raw$ data. We present an improved flat-fielding strategy that limits artefacts when the $\science$ and $\flux$ standard stars are observed on different nights.   The improved $\flux$ standard models and the new grid of anchor points allow to limit artefacts of the response curve correction on, e.g., the shape of the wings of the Balmer lines, from a couple of per cent  of the continuum level to less than 0.5 \% .   We confirm the presence of a radial velocity shift of about 3.5 $\kms$ between the UVB and the VIS arm of $\xshooter$ and that there is no short term variations impacting the RV measurements. RV precision better than 1 $\kms$ can be obtained on sharp telluric lines while RV precision of the order of 2 to 3 $\kms$ are obtained on data with the best $\snr$ . For each target observed by $\xshootu$ , we provide three types of data products: (i) two-dimensional spectra for each UVB and VIS exposure before and after correction for the instrument response;   (ii) one-dimensional UVB and VIS spectra as produced by the $\xshooter$ pipeline  before and after response-correction, as well as after applying various processing, including  absolute flux calibration, telluric removal, normalisation and barycentric correction; and (iii) co-added flux-calibrated and rectified spectra over the full optical range, for which all available $\xshootu$ exposures were combined. For the large majority of the targets, the final signal-to-noise ratio per resolution element  is above 200 in both the UVB and the VIS co-added spectra. The reduced data and advanced scientific data products are made available to the community. Together with the HST UV ULLYSES data, they should enable various science goals, from detailed stellar atmosphere and stellar wind studies, to empirical libraries for population synthesis, to study of the local nebular environment and feedback of massive stars in sub-solar metallicity environments.

</div>

<div id="div_fig1">

<img src="tmp_2402.16987/./FIGURES/fluxCompare.png" alt="Fig16" width="100%"/>

**Figure 16. -** Comparison of the spectrum of AV-43 before slit-loss correction (lower spectrum), after slit-loss and telluric absorption correction (middle spectrum), and after the scaling to the B magnitude and lining up the VIS to the UVB (upper spectrum). The corresponding V and R magnitudes are indicated using a star symbol. Transmission curves are from [ and Bessell (1990)](). (*f:FluxCal*)

</div>
<div id="div_fig2">

<img src="tmp_2402.16987/./FIGURES/QCXSH-U_2dFS-163_COADD1D_UVBA_NORM_bis.png" alt="Fig19.1" width="25%"/><img src="tmp_2402.16987/./FIGURES/QCXSH-U_2dFS-163_COADD1D_UVBB_NORM_bis.png" alt="Fig19.2" width="25%"/><img src="tmp_2402.16987/./FIGURES/QCXSH-U_2dFS-163_COADD1D_VISA_NORM_bis.png" alt="Fig19.3" width="25%"/><img src="tmp_2402.16987/./FIGURES/QCXSH-U_2dFS-163_COADD1D_VISB_NORM_bis.png" alt="Fig19.4" width="25%"/>

**Figure 19. -** Example of automatic normalisation of 2DFS-163 for each range in the UVB and VIS arms. Regions shaded in red and green are regions that are rejected or forcefully included as continuum regions.  (*f:QCnorm*)

</div>
<div id="div_fig3">

<img src="tmp_2402.16987/./FIGURES/EG274_updated_fit_latest.png" alt="Fig4.1" width="50%"/><img src="tmp_2402.16987/./FIGURES/EG21_updated_fit_latest.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Comparison of new and old reference models for  the EG 274 and EG 21 $\flux$ standard stars used by the $\xsh$ootu program.  The top sub-panel of each star show the observed co-added spectrum (in black) and the new and old reference models (in red and blue, respectively).  The bottom sub-panels show the residuals between the observation and each of the reference models using the same color scheme. (*f:FluxModel*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2402.16987"></div>

 marks the moving standard deviation of the their synthetic planets. (*fig:mass-radius*)

</div>
<div id="div_fig3">

<img src="tmp_2402.17592/./Fig/tepcat_mass-separation.png" alt="Fig1" width="100%"/>

**Figure 1. -** Mass-separation diagram for planets around solar-analog stars (definition according to [Lehmann, et. al (2022)](), see \ref{sec:Introduction}). The new planets TOI-2373 b, TOI-2416 b, and TOI-2524 b are shown as big stars. (*fig:mass-sep*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2402.17592"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

377  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
